In [1]:
from mysql_connection import get_cursor
import time
import pandas as pd
import numpy as np

cursor = get_cursor()
# Start the timer
start = time.perf_counter()

## Call get food checks by id with the user id

In [2]:
user_id = (1,)
foodchecks = cursor.callproc("get_foodchecks_by_id", user_id)
checks = []
# stored_results is iterable, have to do this way
for result in cursor.stored_results():
    checks = result.fetchall()[0][1:]

print(checks)

(0, 0, 0, 0, 1, 1, 1, 1, 1)


## Parse all the results, and create the WHERE clause

In [3]:
column_rows = ["IsVegetarian", "IsVegan", "IsHalal", "IsKosher", "HasLactose", "HasNuts", "HasGluten", "HasEgg", "HasSoy"]

def print_food_checks(result) :
    for i in range(len(result)):
        print(f"{column_rows[i]} : {result[i]}")

print_food_checks(checks)

def food_checks_builder(result):
    food_checks = []
    for i in range(len(result)):
        if(result[i]):
            food_checks.append(f"{column_rows[i]}=1")
    return " AND ".join(food_checks)

print("---------------------------------")
print(food_checks_builder(checks))

IsVegetarian : 0
IsVegan : 0
IsHalal : 0
IsKosher : 0
HasLactose : 1
HasNuts : 1
HasGluten : 1
HasEgg : 1
HasSoy : 1
---------------------------------
HasLactose=1 AND HasNuts=1 AND HasGluten=1 AND HasEgg=1 AND HasSoy=1


## Get the user's most liked and disliked items


In [4]:
cursor.execute(f"SELECT FoodTagID FROM FoodOpinionRightSwipePercent WHERE UserID={user_id[0]};")
result = cursor.fetchall()
food_tags = [str(i[0]) for i in result[:5] + result[-5:]]
print(food_tags)

['4', '9', '2', '1', '7', '8', '5', '11', '12', '3']


## Get all the user id's with at least the food checks that you have
### i.e. You are vegetarian... Some one who is vegetarian, egg-free, and lactose-free will also be selected 

In [5]:
cursor.execute(f"SELECT UserID, FoodTagID, RightSwipePercent FROM FoodOpinionRightSwipePercent WHERE UserID IN(SELECT UserID FROM tat.UserFoodCheckView WHERE {food_checks_builder(checks)} AND UserID<>{user_id[0]}) AND FoodTagID IN ({','.join(food_tags)});")
result = cursor.fetchall()
print(pd.DataFrame(result))

   0   1       2
0  2   7  1.9806
1  2   2  1.7647
2  2   9  1.7391
3  2   1  1.6667
4  2   8  0.8000
5  2  12  0.7317
6  2   4  0.5455
7  2   5  0.5000
8  2   3  0.4000
9  2  11  0.3636


In [6]:
tag_dict = {}
cursor.execute(f"SELECT UserID, FoodTagID, RightSwipePercent FROM FoodOpinionRightSwipePercent WHERE UserID={user_id[0]};")
user = cursor.fetchall()
for row in user:
    tag_dict[row[1]] = float(row[2])

print(tag_dict)

{4: 1.8649, 9: 1.8462, 2: 1.7647, 1: 1.6667, 7: 1.5789, 10: 1.5238, 6: 1.0909, 8: 1.0, 5: 0.8889, 11: 0.25, 12: 0.0, 3: 0.0}


In [7]:
ptr = 0
count_dict = {55: 2}
TOLERANCE = 0.3

# Go through all results, and count how many tags are within the tolerance
for row in result:
    user_val = tag_dict[row[1]]
    # Within the user's swipe percentage + / - 0.3
    if user_val - TOLERANCE <= row[2] <= user_val + TOLERANCE:
        try:
            count = count_dict[row[0]]
            count_dict[row[0]] = count + 1
        except:
            count_dict[row[0]] = 1

ids_to_get = []

print(count_dict)
for user in count_dict.items():
    if(user[1] >= 5):
        ids_to_get.append(str(user[0]))

print(ids_to_get)

{55: 2, 2: 5}
['2']


In [8]:
cursor.execute(f"SELECT UserID, FoodTagID, RightSwipePercent FROM FoodOpinionRightSwipePercent WHERE UserID IN ({','.join(ids_to_get)});")
result = cursor.fetchall()
df = pd.DataFrame(result)
df.columns = ["UserID", "FoodTagID", "RightSwipePct"]
similarities = []
for row in df.iterrows():
    id = row[1][0]
    similarities.append(count_dict[id] / 5)

df["Bias"] = similarities
print(df)

    UserID  FoodTagID RightSwipePct  Bias
0        2          7        1.9806   1.0
1        2          2        1.7647   1.0
2        2          9        1.7391   1.0
3        2          1        1.6667   1.0
4        2          6        1.0769   1.0
5        2          8        0.8000   1.0
6        2         12        0.7317   1.0
7        2          4        0.5455   1.0
8        2          5        0.5000   1.0
9        2          3        0.4000   1.0
10       2         11        0.3636   1.0
11       2         10        0.0000   1.0


In [9]:


end = time.perf_counter()
print(f"Done in {end - start:0.4f} seconds")



Done in 0.2224 seconds
